### Часть 1: Агент без памяти (stateless)

In [8]:
from langchain.agents import create_agent
from langchain.messages import HumanMessage
import os
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import InMemorySaver

from dotenv import load_dotenv

load_dotenv()

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise EnvironmentError("Установите OPENROUTER_API_KEY в файле .env")

llm = ChatOpenAI(
    model="google/gemini-2.5-flash",
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY
)

In [4]:
agent = create_agent(model=llm)

question = HumanMessage(content="Привет, меня зовут Сергей. Мой любимый цвет синий")

response = agent.invoke(
    {"messages": [question]}
)
print(response['messages'][1].content)

Приятно познакомиться, Сергей! Синий – отличный выбор, это очень красивый и многогранный цвет. Он часто ассоциируется со спокойствием, мудростью и надежностью. 😊

А есть ли у синего цвета какие-то особые оттенки, которые вам нравятся больше всего? Или, может быть, какие-то ассоциации, которые вы с ним связываете?


In [5]:
question = HumanMessage("Какой мой любимый цвет?")

response = agent.invoke(
    {'messages': [question]}
)

print(response['messages'][1].content)

Так как я не знаю ни Вашего имени, ни какой-либо другой личной информации о Вас, то не могу знать, какой у Вас любимый цвет. 

Но если Вы хотите поделиться им, я с удовольствием узнаю! 🎨


### Часть 2: Агент с памятью (stateful через LangGraph)

Что изменилось:
- Мы передали checkpointer=InMemorySaver().
- Это включает механизм сохранения состояния (checkpointing) из LangGraph.
- Теперь агент может помнить историю сообщений между вызовами — но только в рамках одного "потока" (thread).

#### Конфигурация потока (thread_id)

- thread_id — это идентификатор диалога (например, ID пользователя или сессии).
- Все вызовы с thread_id="1" будут делиться одной памятью.
- Если создать thread_id="2" — это будет отдельный диалог, с нуля.

Это аналог сеанса (session) в веб-приложениях.

In [10]:
agent = create_agent(
    model=llm,
    checkpointer=InMemorySaver()
)

question = HumanMessage(content="Меня зовут Сергей. Мой любимый цвет синий")
config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [question]},
    config
)

print(response['messages'][1].content)

Приятно познакомиться, Сергей! Синий – отличный выбор! Это очень интересный и многогранный цвет. Он может быть спокойным и безмятежным, как летнее небо, а может быть глубоким и таинственным, как океан.

Что вам особенно нравится в синем цвете? Может быть, его ассоциации с чем-то конкретным, или просто его визуальное воздействие?


In [11]:
question = HumanMessage(content="What's my favourite colour?")

response = agent.invoke(
    {"messages": [question]},
    config,  
)

print(response['messages'][-1].content)

Your favourite colour is **blue**!
